In [2]:
import os
import sys
sys.path.insert(0,'../')
from libs.ssl_dataloader import *
from libs.ssl_model import *
from libs.ssl_utils import *
from libs.eeg_utils import *
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows)
from braindecode.datasets import BaseDataset, BaseConcatDataset, WindowsDataset
from braindecode.preprocessing.windowers import EEGWindowsDataset
%load_ext autoreload
%autoreload 2
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
random_state = 87
n_jobs = 1

In [4]:
datasets = []
hbn_datasets = ['ds005514','ds005512','ds005511','ds005510','ds005509','ds005508','ds005507','ds005506','ds005505']

In [5]:
ds1 = HBNDataset('ds005505', tasks=['RestingState'], subjects=['NDARRD326KB9', 'NDARZD415ZZ1', 'NDARXH597ML1'])
ds2 = HBNDataset('ds005510', tasks=['RestingState'], subjects=['NDARAE877NER', 'NDARYW984FLT', 'NDARFX710UZA'])

Loading /mnt/nemar/openneuro/ds005505/sub-NDARRD326KB9/eeg/sub-NDARRD326KB9_task-RestingState_eeg.set


/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)


Loading /mnt/nemar/openneuro/ds005505/sub-NDARZD415ZZ1/eeg/sub-NDARZD415ZZ1_task-RestingState_eeg.set


/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)


Loading /mnt/nemar/openneuro/ds005505/sub-NDARXH597ML1/eeg/sub-NDARXH597ML1_task-RestingState_eeg.set


...

/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)


Loading /mnt/nemar/openneuro/ds005510/sub-NDARYW984FLT/eeg/sub-NDARYW984FLT_task-RestingState_eeg.set


/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)


Loading /mnt/nemar/openneuro/ds005510/sub-NDARFX710UZA/eeg/sub-NDARFX710UZA_task-RestingState_eeg.set


/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)
/home/dung/eeg-ssl/notebooks/../libs/ssl_dataloader.py:223: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(raw_file, preload=preload)


In [6]:
all_ds = BaseConcatDataset([ds1, ds2])

In [7]:
from numpy import multiply
high_cut_hz = 30
# Factor to convert from V to uV
factor = 1e6
preprocessors = [
    Preprocessor(lambda data: multiply(data, factor)),  # Convert from V to uV
    Preprocessor('filter', l_freq=None, h_freq=high_cut_hz, n_jobs=n_jobs)
]

# Transform the data
preprocess(all_ds, preprocessors)

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 221 samples (0.442 s)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 221 samples (0.442 s)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter param

In [8]:
window_len_s = 30
fs = all_ds.datasets[0].raw.info['sfreq']
window_len_samples = int(fs * window_len_s)
window_stride_samples = int(fs * 4)
# window_stride_samples = int(fs * window_len_s)
windows_ds = create_fixed_length_windows(
    all_ds, start_offset_samples=0, stop_offset_samples=None,
    window_size_samples=window_len_samples,
    window_stride_samples=window_stride_samples, drop_last_window=True,
    preload=False)

In [9]:
from sklearn.preprocessing import scale as standard_scale

preprocess(windows_ds, [Preprocessor(standard_scale, channel_wise=True)])

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:265: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:265: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:265: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:265: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard devia

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from braindecode.datasets import BaseConcatDataset

subjects = np.unique(windows_ds.description['subject'])
subj_train, subj_test = train_test_split(
    subjects, test_size=0.4, random_state=random_state)
subj_valid, subj_test = train_test_split(
    subj_test, test_size=0.5, random_state=random_state)

In [11]:
class RelativePositioningDataset(BaseConcatDataset):
    """BaseConcatDataset with __getitem__ that expects 2 indices and a target.
    """

    def __init__(self, list_of_ds):
        super().__init__(list_of_ds)
        self.return_pair = True

    def __getitem__(self, index):
        if self.return_pair:
            ind1, ind2, y = index
            return (super().__getitem__(ind1)[0],
                    super().__getitem__(ind2)[0]), y
        else:
            return super().__getitem__(index)

    @property
    def return_pair(self):
        return self._return_pair

    @return_pair.setter
    def return_pair(self, value):
        self._return_pair = value


split_ids = {'train': subj_train, 'valid': subj_valid, 'test': subj_test}
splitted = dict()
for name, values in split_ids.items():
    splitted[name] = RelativePositioningDataset(
        [ds for ds in windows_ds.datasets
         if ds.description['subject'] in values])


In [12]:
splitted['train'].get_metadata()

,i_window_in_trial,i_start_in_trial,i_stop_in_trial,target,task,session,run,subject,sfreq
0,0,0,15000,-1,RestingState,,,NDARZD415ZZ1,500
1,1,2000,17000,-1,RestingState,,,NDARZD415ZZ1,500
2,2,4000,19000,-1,RestingState,,,NDARZD415ZZ1,500
3,3,6000,21000,-1,RestingState,,,NDARZD415ZZ1,500
4,4,8000,23000,-1,RestingState,,,NDARZD415ZZ1,500
...,...,...,...,...,...,...,...,...,...
77,77,154000,169000,-1,RestingState,,,NDARYW984FLT,500
78,78,156000,171000,-1,RestingState,,,NDARYW984FLT,500
79,79,158000,173000,-1,RestingState,,,NDARYW984FLT,500
80,80,160000,175000,-1,RestingState,,,NDARYW984FLT,500


In [13]:
from braindecode.samplers import RelativePositioningSampler

sfreq = 500
tau_pos, tau_neg = int(sfreq * 60), int(sfreq * 2 * 60)
n_examples_train = 250 * len(splitted['train'].datasets)
n_examples_valid = 250 * len(splitted['valid'].datasets)
n_examples_test = 250 * len(splitted['test'].datasets)

train_sampler = RelativePositioningSampler(
    splitted['train'].get_metadata(), tau_pos=tau_pos, tau_neg=tau_neg,
    n_examples=n_examples_train, same_rec_neg=True, random_state=random_state)
valid_sampler = RelativePositioningSampler(
    splitted['valid'].get_metadata(), tau_pos=tau_pos, tau_neg=tau_neg,
    n_examples=n_examples_valid, same_rec_neg=True,
    random_state=random_state).presample()
test_sampler = RelativePositioningSampler(
    splitted['test'].get_metadata(), tau_pos=tau_pos, tau_neg=tau_neg,
    n_examples=n_examples_test, same_rec_neg=True,
    random_state=random_state).presample()

In [ ]:
import torch
from torch import nn
from braindecode.util import set_random_seeds
from braindecode.models import SleepStagerChambon2018

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.benchmark = False
# Set random seed to be able to roughly reproduce results
# Note that with cudnn benchmark set to True, GPU indeterminism
# may still make results substantially different between runs.
# To obtain more consistent results at the cost of increased computation time,
# you can set `cudnn_benchmark=False` in `set_random_seeds`
# or remove `torch.backends.cudnn.benchmark = True`
set_random_seeds(seed=random_state, cuda=device == 'cuda')

# Extract number of channels and time steps from dataset
n_channels, input_size_samples = windows_ds[0][0].shape
emb_size = 100
classes = list(range(5))

emb = SleepStagerChambon2018(
    n_channels,
    sfreq,
    n_outputs=emb_size,
    n_conv_chs=16,
    n_times=input_size_samples,
    dropout=0,
    apply_batch_norm=True,
)


class ContrastiveNet(nn.Module):
    """Contrastive module with linear layer on top of siamese embedder.

    Parameters
    ----------
    emb : nn.Module
        Embedder architecture.
    emb_size : int
        Output size of the embedder.
    dropout : float
        Dropout rate applied to the linear layer of the contrastive module.
    """

    def __init__(self, emb, emb_size, dropout=0.5):
        super().__init__()
        self.emb = emb
        self.clf = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(emb_size, 1)
        )

    def forward(self, x):
        x1, x2 = x
        z1, z2 = self.emb(x1), self.emb(x2)
        return self.clf(torch.abs(z1 - z2)).flatten()


model = ContrastiveNet(emb, emb_size).to(device)

In [15]:
import os

from skorch.helper import predefined_split
from skorch.callbacks import Checkpoint, EarlyStopping, EpochScoring
from braindecode import EEGClassifier

lr = 5e-3
batch_size = 12  # 512 if data large enough
n_epochs = 25
num_workers = 0 if n_jobs <= 1 else n_jobs

cp = Checkpoint(dirname='', f_criterion=None, f_optimizer=None, f_history=None)
early_stopping = EarlyStopping(patience=10)
train_acc = EpochScoring(
    scoring='accuracy', on_train=True, name='train_acc', lower_is_better=False)

callbacks = [
    ('cp', cp),
    ('patience', early_stopping),
    ('train_acc', train_acc),
]

clf = EEGClassifier(
    model,
    criterion=torch.nn.BCEWithLogitsLoss,
    optimizer=torch.optim.Adam,
    max_epochs=n_epochs,
    iterator_train__shuffle=False,
    iterator_train__sampler=train_sampler,
    iterator_valid__sampler=valid_sampler,
    iterator_train__num_workers=num_workers,
    iterator_valid__num_workers=num_workers,
    train_split=predefined_split(splitted['valid']),
    optimizer__lr=lr,
    batch_size=batch_size,
    callbacks=callbacks,
    device=device,
    classes=classes,
)
# Model training for a specified number of epochs. `y` is None as it is already
# supplied in the dataset.
clf.fit(splitted['train'], y=None)
clf.load_params(checkpoint=cp)  # Load the model with the lowest valid_loss

os.remove('./params.pt')  # Delete parameters file

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      dur
-------  -----------  ------------  -----------  ------------  ----  -------
      1       0.8038        0.5658       0.6880        0.5976     +  42.1334
      2       0.8777        0.3160       0.7400        0.5794     +  43.0816
      3       0.8992        0.2981       0.6720        0.5500     +  49.1422
      4       0.9046        0.2479       0.6240        0.8002        48.7186
      5       0.9301        0.2285       0.5760        1.1914        49.1837
      6       0.9194        0.2539       0.6280        0.7326        49.1231
      7       0.9341        0.1808       0.7360        0.4952     +  49.0041
      8       0.9167        0.2594       0.7600        0.5544        48.7554
      9       0.9435        0.1749       0.8160        0.4511     +  48.8163
     10       0.9570        0.1334       0.6560        0.8971        49.1937
     11       0.9395        0.1830       0.6760        0.7021        48.7823

# Torch Lightning

In [37]:
import lightning as L
import torch
from torch import nn
from braindecode.util import set_random_seeds
from braindecode.models import SleepStagerChambon2018
# Extract number of channels and time steps from dataset
n_channels, input_size_samples = windows_ds[0][0].shape
emb_size = 100
classes = list(range(5))

emb = SleepStagerChambon2018(
    n_channels,
    sfreq,
    n_outputs=emb_size,
    n_conv_chs=16,
    n_times=input_size_samples,
    dropout=0,
    apply_batch_norm=True,
)

# define the LightningModule
class LitSSL(L.LightningModule):
    def __init__(self, emb, emb_size, dropout=0.5):
        super().__init__()
        self.emb = emb
        self.clf = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(emb_size, 1)
        )

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        X, y = batch
        x1, x2 = X[0], X[1]
        z1, z2 = self.emb(x1), self.emb(x2)
        loss = nn.functional.binary_cross_entropy_with_logits(self.clf(torch.abs(z1 - z2)).flatten(), y)

        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

model =  LitSSL(emb, emb_size)

In [40]:
train_loader = DataLoader(splitted['train'], sampler=train_sampler, batch_size=12, num_workers=4)
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(limit_train_batches=100, max_epochs=1, accelerator='gpu')
trainer.fit(model=model, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                   | Params | Mode 
--------------------------------------------------------
0 | emb  | SleepStagerChambon2018 | 704 K  | train
1 | clf  | Sequential             | 101    | train
--------------------------------------------------------
704 K     Trainable params
0         Non-trainable params
704 K     Total params
2.817     Total estimated model params size (MB)
17        Modules in train mode
0         Modules in eval mode


Epoch 0:   0%|          | 0/63 [00:00<?, ?it/s] 

Epoch 0: 100%|██████████| 63/63 [00:23<00:00,  2.68it/s, v_num=5]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 63/63 [00:23<00:00,  2.67it/s, v_num=5]
